In [2]:
import streamlit as st

from PIL import Image
import numpy as np
import cv2

DEMO_IMAGE = 'stand.jpg'

BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]


width = 368
height = 368
inWidth = width
inHeight = height

net = cv2.dnn.readNetFromTensorflow("graph_opt.pb")



def main():
    st.title("Human Pose Estimation OpenCV")
    
    st.text('Make Sure you have a clear image with all the parts clearly visible')
    
    img_file_buffer = st.file_uploader("Upload an image, Make sure you have a clear image", type=[ "jpg", "jpeg",'png'])
    
    if img_file_buffer is not None:
        image = np.array(Image.open(img_file_buffer))
    
    else:
        demo_image = DEMO_IMAGE
        image = np.array(Image.open(demo_image))
        
    st.subheader('Original Image')
    st.image(
        image, caption=f"Original Image", use_column_width=True
    ) 
    
    thres = st.slider('Threshold for detecting the key points',min_value = 0,value = 20, max_value = 100,step = 5)
    
    thres = thres/100
    
    @st.cache
    def poseDetector(frame):
        frameWidth = frame.shape[1]
        frameHeight = frame.shape[0]
        
        net.setInput(cv2.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), 
                                           swapRB=True, crop=False))
        
        out = net.forward()
        out = out[:, :19, :, :]
        
        assert(len(BODY_PARTS) == out.shape[1])
        
        points = []
        for i in range(len(BODY_PARTS)):
            # Slice heatmap of corresponging body's part.
            heatMap = out[0, i, :, :]
    
            _, conf, _, point = cv2.minMaxLoc(heatMap)
            x = (frameWidth * point[0]) / out.shape[3]
            y = (frameHeight * point[1]) / out.shape[2]
            points.append((int(x), int(y)) if conf > thres else None)
            
            
        for pair in POSE_PAIRS:
            partFrom = pair[0]
            partTo = pair[1]
            assert(partFrom in BODY_PARTS)
            assert(partTo in BODY_PARTS)
    
            idFrom = BODY_PARTS[partFrom]
            idTo = BODY_PARTS[partTo]
    
            if points[idFrom] and points[idTo]:
                cv2.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
                cv2.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)
                cv2.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)
                
                
        t, _ = net.getPerfProfile()
        
        return frame
    
    
    output = poseDetector(image)
    
    
    st.subheader('Positions Estimated')
    st.image(
           output, caption=f"Positions Estimated", use_column_width=True)
        
    st.markdown('''
              # Author \n 
                 Hey this is ** Pavan Kunchala ** I hope you like the application \n
                I am looking for ** Collabration ** or ** Freelancing ** in the field of ** Deep Learning ** and 
                ** Computer Vision ** \n
                If you're interested in collabrating you can mail me at ** pavankunchalapk@gmail.com ** \n
                You can check out my ** Linkedin ** Profile from [here](https://www.linkedin.com/in/pavan-kumar-reddy-kunchala/) \n
                You can check out my ** Github ** Profile from [here](https://github.com/Pavankunchala) \n
                You can also check my technicals blogs in ** Medium ** from [here](https://pavankunchalapk.medium.com/) \n
                If you are feeling generous you can buy me a cup of ** coffee ** from [here](https://www.buymeacoffee.com/pavankunchala)
                 
                ''')
if __name__ == '__main__':
    main()


InternalHashError: module '__main__' has no attribute '__file__'

While caching the body of `poseDetector()`, Streamlit encountered an
object of type `builtins.function`, which it does not know how to hash.

**In this specific case, it's very likely you found a Streamlit bug so please
[file a bug report here.]
(https://github.com/streamlit/streamlit/issues/new/choose)**

In the meantime, you can try bypassing this error by registering a custom
hash function via the `hash_funcs` keyword in @st.cache(). For example:

```
@st.cache(hash_funcs={builtins.function: my_hash_func})
def my_func(...):
    ...
```

If you don't know where the object of type `builtins.function` is coming
from, try looking at the hash chain below for an object that you do recognize,
then pass that to `hash_funcs` instead:

```
Object of type builtins.function: <function main.<locals>.poseDetector at 0x000001C6CDB1C820>
```

Please see the `hash_funcs` [documentation]
(https://docs.streamlit.io/en/stable/caching.htm